In [ ]:
import csv
from datetime import datetime, timedelta
from time import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns

%matplotlib inline
pd.optionmax_rows = 100

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.metrics import classification_report, confusion_matrix 
import sklearn.svm
from sklearn.metrics import f1_score


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.semi_supervised import SelfTrainingClassifier, LabelPropagation

In [ ]:
clustered_sessions_20_21 = pd.read_csv("../Clustered_Sessions_FCM.csv")
clustered_sessions_20_21['session_number'] +='-20_21'

clustered_sessions_22 = pd.read_csv("../Spring_22/Clustered_Sessions_FCM.csv")
clustered_sessions_22['session_number'] +='-22'

clustered_sessions_all = pd.concat([clustered_sessions_20_21,clustered_sessions_22])
clustered_sessions_all.columns # Use it to get if a session is credit seeking or normal

In [ ]:
clustered_sessions_all = clustered_sessions_all.fillna(0)
clustered_sessions_all # We don't care here about whether 2 interactions or not, bcz the model deosn't cluster unless it's 3 or more

# Semi Supervised Learning

In [ ]:
# What confidence level do we achieve
# How much is the accuracy on the proportions that we kept unlabeled
# Which model we used
# How long it took us to itereate
# The attributes we are training on
# What is the benefit of adding that to the model

# Do we need the PCA steps again?

In [ ]:
len(clustered_sessions_all)

In [ ]:
data = clustered_sessions_all.iloc[:,5:-1].to_numpy()

In [ ]:
target_original = clustered_sessions_all['cluster'].to_numpy().copy()


target_modified = clustered_sessions_all['cluster'].to_numpy().copy()
rng = np.random.RandomState(2)
unl_idxs = rng.rand(len(target_modified)) < 0.95 # It means you will create random numbers, we don't know the values, but they are 12645 values, and all range from 0 to 1 (decimals)
target_modified[unl_idxs] = -1 # Anything that should be unlabled, according to the model will have to be -1, We used 'unl_idxs' as a mask

In [ ]:
n_labeled = len(target_modified) - len(target_modified[unl_idxs])
n_unlabeled =  len(target_modified[unl_idxs])
print("Labeled:", n_labeled, "Unlabeled:", n_unlabeled, " -- % of Labeled", n_labeled/ len(target_modified),  "-- % of Unlabeled", n_unlabeled/ len(target_modified), )

In [ ]:
clf = GradientBoostingClassifier()
lbl = SelfTrainingClassifier(clf,max_iter=20)
lbl.fit(data, target_modified)
# print ("Semi_supervised labels:\n", y_ pred)
y_pred = lbl.predict(data[unl_idxs]) 
print(classification_report(target_original[unl_idxs], y_pred))  
             

In [ ]:
f1_score(target_original[unl_idxs], y_pred, average='micro', )


In [ ]:
y_pred[:100]

In [ ]:
len(y_pred)

In [ ]:
len(data[unl_idxs])

In [ ]:
target_original[unl_idxs][:10]

In [ ]:
y_pred[unl_idxs][:10]

In [ ]:
lbl.__dict__

In [ ]:
label_prop_model = LabelPropagation()
lbl.fit(data, target_modified)
# print ("Semi_supervised labels:\n", y_ pred)
y_pred = lbl.predict(data) 
print(classification_report(target_original[unl_idxs], y_pred[unl_idxs]))  
             

In [ ]:
# What is the lowest level ?
# What is the chapter names from 1 to 5
# Only get those framesets, and train on 20,21, and predict on 22, then try to see what F1 can be improved if not already in shaa allah

In [ ]:
Frameset_Questions_Coarse_Grained = pd.read_csv('../Frameset_Questions_Coarse_Grained.csv')

new_df = pd.merge(clustered_sessions_20_21, Frameset_Questions_Coarse_Grained[['frame_name_22','Chapter']],left_on = 'curr_frameset_name', right_on = 'frame_name_22', how='inner')

# new_df = new_df[new_df.Chapter<=5]


new_df = new_df.pivot_table(columns='curr_frameset_name',index=['user_id','cluster'],aggfunc=len,values='u').fillna(0).reset_index()



In [ ]:
# Frameset_Questions_Coarse_Grained.columns, len(new_df.columns)

In [ ]:
# len(Frameset_Questions_Coarse_Grained['frame_name_22'].drop_duplicates().unique())
# Frameset_Questions_Coarse_Grained = Frameset_Questions_Coarse_Grained.drop_duplicates(subset=['frame_name_22'])

In [ ]:
chapter_names = ['Ch1: Introduction', 'Ch2: Mathematical Background', 'Ch3: Finite Acceptors', 'Ch4: Regular Languages', 'Ch5: Identifying Non-regular Languages', 'Ch6: Context-Free Grammars and Languages',
'Ch7: Pushdown Automata', 'Ch8: Properties of Context-free Languages', 'Ch9: Turing Machines', 'Ch10: Parsing', 'Ch11: Limits to Computing']

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
Y = [2,8,8,22,12,11,10,7,5,0,9]
gfg = sns.barplot(y=Y, x=chapter_names, color='purple')#,kind="kde")
gfg.set(
    # xlabel="Module Number",
    # ylabel="Number of Framesets",
    # title="Frequency of Framesets Attempted Per Sutdent",
    # xticks=(range(0, max(n_frames_per_student) + 3, 2)),
)
ax.set_xlabel("Chapter name", fontsize = 12,labelpad=20) 
ax.set_ylabel("Number of framesets",  fontsize = 12) 
ax.set_xticklabels(chapter_names, rotation = 90)
plt.bar_label(gfg.containers[0])
plt.show()
# The x-axis is # of framesets attempted, The y-axis is the # of students who attempted the same # of framesets
fig.savefig("Chapters-Division.pdf", facecolor="white",bbox_inches="tight")


In [ ]:
crsk_count  = new_df[new_df.cluster=='Credit Seeking'].set_index('user_id').drop('cluster',axis=1)
normal_count  = new_df[new_df.cluster=='Normal'].set_index('user_id').drop('cluster',axis=1)


agg_count = pd.DataFrame(columns=crsk_count.columns,index=crsk_count.index).fillna(0)

In [ ]:
# Let's aggregate by Chapter Maybe??

In [ ]:
try:
    for user_id in agg_count.index:
        for chapter_number in agg_count.columns:
            crsk = crsk_count.loc[user_id, chapter_number] if user_id in crsk_count.index else 0
            nrml = normal_count.loc[user_id, chapter_number]  if user_id in normal_count.index else 0
            if crsk + nrml ==0:
                agg_count.loc[user_id, chapter_number] = 0
            else:
                agg_count.loc[user_id, chapter_number] = nrml  / (nrml + crsk)
except Exception as E:
    print([user_id, chapter_number] )

agg_count = agg_count.fillna(0)

In [ ]:
# The value of each cell in the spreadsheet represents the proportion of the normal attempts for this frameset.
# For example, if a student has attempted the frameset “GrammarIntroFS” 3 times (3 sessions), and he twice did it as credit-seeking, and once as a normal attempt, 
# then the value of the corresponding cell will be (⅓). 

# Using this spreadsheet, I was able to reach eventually an accuracy up to 68.75% by using RandomForestClassifier. The F-score for each class is the following: [A, B, C]
# I believe that this accuracy is pretty much high given that these framesets were totally optional. 


In [ ]:
# grades_22 = pd.read_csv("../Spring_22/all_scores.csv")[['odsa_ID','Final Score']]
grades_20_21 = pd.read_excel("../Grades/Book1.xlsx")[['odsa_ID','Final Score']]
grades_20_21.columns

In [ ]:
grades_20_21[grades_20_21.odsa_ID==6728]


In [ ]:
# https://courses.cs.vt.edu/~cs1604/grading.html

letter_grade = []
for grade in grades_20_21["Final Score"]:
    if grade >= 90:
        letter_grade.append("A")
    elif grade >= 80:
        letter_grade.append("B")
    elif grade >= 70:
        letter_grade.append("C")
    elif grade >= 63.6:
        letter_grade.append("D")
    else:
        letter_grade.append("F")

grades_20_21["Letter Grade"] = letter_grade

In [ ]:
grades_20_21["Letter Grade"].value_counts()

In [ ]:
grades_20_21["Final Score"].plot(kind='box'),grades_20_21["Final Score"].describe()

In [ ]:
# Y = 
X ='Final Score'
# res = stats.linregress(sessions_attributes_fine_grained_multi[X], sessions_attributes_fine_grained_multi[Y])
# rvalue, pvalue = stats.pearsonr(x=sessions_attributes_fine_grained_multi[X], y=sessions_attributes_fine_grained_multi[Y])
# print(f"R: {res.rvalue}")
# print("P-value", pvalue)
fig, axs = plt.subplots(figsize=(10,10))

gfg = sns.ecdfplot(data=grades_20_21, x=X)

axs.set_xlabel( xlabel='Overall score', fontsize = 12 ) 
axs.set_ylabel( ylabel='Proportion', fontsize = 12) 
plt.legend(loc='upper right')
plt.show()
fig.savefig("Overall-Score-CDF.pdf", facecolor="white",dpi=500)

# It means whether there are multiple number of choices or not, we may classifiy it as credit seeking or not

In [ ]:
grades_20_21.columns

In [ ]:
79.992500-1.5*(90.945-79.99250)

In [ ]:
merged_df = pd.merge(grades_20_21, agg_count, how='inner', left_on='odsa_ID', right_index=True)
merged_df = merged_df[merged_df["Letter Grade"] != 'D'] # Because unbalanced with D and F. So, I am dropping them. 
merged_df = merged_df[merged_df["Letter Grade"] != 'F']

In [ ]:
all_y = merged_df["Letter Grade"].to_numpy()
all_y_numerical = merged_df["Final Score"].to_numpy()
all_X = merged_df.iloc[:,3:] # Skips the other columns that come from the merge with Grades csv, -1 to skip the sum col. originally use [:,8:] for new_df_crsk_count, new_df_normal_count but not new_df_agg

In [ ]:
merged_df["Letter Grade"].value_counts()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

In [ ]:
# from sklearn import svm
# all_X_normalized =  StandardScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler

# X_train, X_test, y_train, y_test = train_test_split(
#     all_X, all_y, test_size=0.2, shuffle=True,  )


# clf = svm.SVC(max_iter=10000)
# clf.fit(X_train, y_train)

# cv_score = np.mean(cross_val_score(clf, X_train, y_train, cv=10))
# test_score = clf.score(X_test, y_test)

# y_pred = clf.predict(X_test)

# print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
# # print(classification_report(y_test, y_pred))

# # The point of this method is that since only 1/10 of your data is in the test set, it is unlikely that all your minority class samples end up in the test set.

In [ ]:
from sklearn import svm
all_X_normalized =  StandardScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler

# pca_model = PCA(n_components=3).fit(all_X_normalized)
# all_X_normalized = pca_model.transform(all_X_normalized)
# print(pca_model.explained_variance_ratio_ , sum(pca_model.explained_variance_ratio_)) #69.6
sm = RandomOverSampler()#
all_X_resambled, all_y_resambled = sm.fit_resample(all_X_normalized, all_y)

k = 5
kf = KFold(n_splits=k, random_state=None)
model = svm.SVC(max_iter=10000)
 
result = cross_val_score(model , all_X_resambled, all_y_resambled, cv = k)
print(np.mean(result))
# 0.717983193277311

In [ ]:
# from sklearn import svm
# all_X_normalized =  StandardScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler

# # pca_model = PCA(n_components=3).fit(all_X_normalized)
# # all_X_normalized = pca_model.transform(all_X_normalized)
# # print(pca_model.explained_variance_ratio_ , sum(pca_model.explained_variance_ratio_)) #69.6

# X_train, X_test, y_train, y_test = train_test_split(
#     all_X_normalized, all_y, test_size=0.2, shuffle=True,  )


# sm = SMOTE('all')#
# X_train, y_train, = sm.fit_resample(X_train, y_train)

# clf = LogisticRegression(max_iter=10000)
# clf.fit(X_train, y_train)

# cv_score = np.mean(cross_val_score(clf, X_train, y_train, cv=10))
# test_score = clf.score(X_test, y_test)

# y_pred = clf.predict(X_test)

# print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
# print(classification_report(y_test, y_pred))

In [ ]:
from sklearn import svm
all_X_normalized =  StandardScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler

# pca_model = PCA(n_components=3).fit(all_X_normalized)
# all_X_normalized = pca_model.transform(all_X_normalized)
# print(pca_model.explained_variance_ratio_ , sum(pca_model.explained_variance_ratio_)) #69.6
sm = RandomOverSampler()#
all_X_resambled, all_y_resambled, = sm.fit_resample(all_X_normalized, all_y)

k = 5
kf = StratifiedKFold(n_splits=k, random_state=None)
model = LogisticRegression(solver= 'liblinear')
 
result = cross_val_score(model , all_X_resambled, all_y_resambled, cv = kf)
print(np.mean(result))

In [ ]:
len(y_pred)

In [ ]:
# To report: 
# Using All framesets that were included, we use SVC and we get: cross-val score of 0.727, and a test score of 0.743

In [ ]:
sm.sampling_strategy

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline


all_X_normalized =  StandardScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(all_X_normalized,
                                                    all_y,
                                                    test_size=0.2,
                                                    stratify=all_y,
                                                    random_state=11)


pipeline = imbpipeline(steps = [['smote', SMOTE(sampling_strategy='all', random_state=11)],
                                ['classifier', LogisticRegression()]])

# stratified_kfold = StratifiedKFold(n_splits=3,
#                                        shuffle=True,
#                                        random_state=11)
    
param_grid = [
    { 
     'classifier' : [LogisticRegression()],       'classifier__penalty' : ['l1', 'l2'],      'classifier__C' : np.logspace(-4, 4, 20),    'classifier__solver' : ['liblinear'],
       # 'smote__sampling_strategy':['minority', 'not minority', 'not majority', ]
    },
       { 
     'classifier' : [ LinearSVC(max_iter=3000)],     
       },
    { 
     'classifier' : [RandomForestClassifier()],     'classifier__n_estimators' : list(range(10,101,10)), 'classifier__bootstrap': [True],     'classifier__max_depth': [80, 90, 100, 110], 
    'classifier__min_samples_leaf': [3, 4, 5], 'classifier__min_samples_split': [8, 10, 12], # 'smote__sampling_strategy':['minority', 'not minority', 'not majority', 'all']
   } ,]

 # 'classifier__max_features': [2, 3],
        
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           # scoring='roc_auc',
                           cv=5,
                           n_jobs=-1)
 
sm = RandomOverSampler()
all_X_resambled, all_y_resambled, = sm.fit_resample(all_X_normalized, all_y)

grid_search.fit(all_X_resambled, all_y_resambled,)
cv_score = grid_search.best_score_ # This is already the cv score, and I will include all the data into the cv and nothing for test score
print(cv_score)
# test_score = grid_search.score(X_test, y_test)
# print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')
# print(classification_report(y_test, y_pred))

In [ ]:
grid_search.best_estimator_

In [ ]:
# Regression Model

from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
all_X_normalized =  MinMaxScaler().fit_transform(all_X) # StandardScaler, MinMaxScaler
all_Y_normalized =  MinMaxScaler().fit_transform(all_y_numerical.reshape(-1, 1)) # StandardScaler, MinMaxScaler

# pca_model = PCA(n_components=3).fit(all_X_normalized)
# all_X_normalized = pca_model.transform(all_X_normalized)
# print(pca_model.explained_variance_ratio_ , sum(pca_model.explained_variance_ratio_)) #69.6

X_train, X_test, y_train, y_test = train_test_split(
    all_X, all_y_numerical, test_size=0.2, shuffle=True,  )

# sm = RandomOverSampler()#
# X_train  = sm.fit_resample(X_train )


krr = svm.SVR() 
krr.fit(X_train, y_train.ravel())
print(krr.score(X_test, y_test.ravel()))
y_pred = krr.predict(X_test)

# A constant model that always predicts the expected value of y, disregarding the input features, would get a  score of 0.0.

In [ ]:
print(list(zip(y_pred, y_test)))

In [ ]:
# # Create a feature attribute

# new_df_crsk_count = pd.DataFrame( columns= ['UserID']+framset_names) 
# new_df_crsk_count['UserID'] = userIDs
# new_df_crsk_count.set_index( 'UserID', inplace=True) 
# new_df_crsk_count = new_df_crsk_count.fillna(0)

# new_df_normal_count = new_df_crsk_count.copy()

# new_df_agg = new_df_crsk_count.copy()

In [ ]:
# for session in sessions_df.iterrows():
#     if session[1].Cluster == 'Credit Seeking':
#         new_df_crsk_count.loc[session[1].UserID][session[1].FramesetName]+=1
        
# for session in sessions_df.iterrows():
#     if session[1].Cluster == 'Normal':
#         new_df_normal_count.loc[session[1].UserID][session[1].FramesetName]+=1
        
# # We want the values to be Percentage of credit seeking of each frameset, instead of just raw values of credit seeking

# for row_id in range(len(new_df_normal_count)):
#     for col_id in range(len(new_df_normal_count.columns)):
#         new_df_agg.iloc[row_id, col_id] = new_df_normal_count.iloc[row_id, col_id] / (new_df_crsk_count.iloc[row_id, col_id]+new_df_normal_count.iloc[row_id, col_id])
        
# new_df_agg = new_df_agg.fillna(0)
        
# # We want the values to be Percentage of credit seeking of each frameset, instead of just raw values of credit seeking

# new_df_normal_count["CrSk Sum"] =  new_df_normal_count.sum(axis=1)
# new_df_normal_count["Normal Sum"] =  new_df_normal_count.sum(axis=1)

# new_df_crsk_count.to_csv("new_df_crsk_count.csv")
# new_df_normal_count.to_csv("new_df_normal_count.csv")
# new_df_agg.to_csv("new_df_agg.csv")

In [ ]:
# From https://udc.vt.edu/irdata/data/courses/grades
# In Fall 2020, we have 70 enrollments 
# In Spring 2021, we have 65 enrollments
# In Spring 2022, we have 75 enrollments

# So having a file "Coarse-grained_Clustered_Sessions.csv" means 
# there's something wrong and we must remove students/accounts to sum up to 70+65 = 135. in Fall'20 and Spring'21. 
# For example, 1270 is Mostafa Mohammed 

# Correctly in "Grades_Users.csv", we have 133 students, and maybe 2 has dropped. 

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.metrics import classification_report, confusion_matrix 
import sklearn.svm

In [ ]:
# When I report the model I used, write clearly why exactly this model, not any other one

In [ ]:
# new_df_agg  = pd.read_csv('new_df_agg.csv')

In [ ]:
# Is there any student who has been registered in two semesters ??

In [ ]:
grades_22 = pd.read_csv("../Spring_22/all_scores.csv")


In [ ]:
grades_20_21 = pd.read_excel("../Grades/Book1.xlsx")
grades_20_21.columns

In [ ]:
grades_all

In [ ]:
# There is no duplicate users between any of the three semester الحمدلله

userIDs_22= grades_22.odsa_ID.unique()
userIDs_20_21 = grades_20_21['OpenDSA ID'].unique()

for user in userIDs_22:
    if user in userIDs_20_21:
        print(user)
print('-'*100)      
for user in userIDs_20_21:
    if user in userIDs_22:
        print(user)

In [ ]:
# https://courses.cs.vt.edu/~cs1604/grading.html

letter_grade = []
for grade in grades_20_21["Final Score"]:
    if grade >= 90:
        letter_grade.append("A")
    elif grade >= 80:
        letter_grade.append("B")
    elif grade >= 70:
        letter_grade.append("C")
    elif grade >= 60:
        letter_grade.append("D")
    else:
        letter_grade.append("F")

grades_20_21["Letter Grade"] = letter_grade

In [ ]:
grades_20_21["Letter Grade"].describe()

In [ ]:
new_df_agg.columns

In [ ]:
merged_df = pd.merge(grades_20_21[['OpenDSA ID',"Letter Grade"]], new_df_agg, how='inner', left_on='OpenDSA ID', right_on='UserID')
merged_df.set_index( 'UserID', inplace=True) 
merged_df = merged_df[merged_df["Letter Grade"] != 'D'] # Because unbalanced with D and F. So, I am dropping them. 
merged_df = merged_df[merged_df["Letter Grade"] != 'F'] # Because unbalanced with D and F. So, I am dropping them. 

In [ ]:
all_y = merged_df["Letter Grade"].to_numpy()
all_X = merged_df.iloc[:,2:] # Skips the other columns that come from the merge with Grades csv, -1 to skip the sum col. originally use [:,8:] for new_df_crsk_count, new_df_normal_count but not new_df_agg

In [ ]:
all_X.shape

In [ ]:
print(merged_df["Letter Grade"].value_counts())
merged_df["Letter Grade"].value_counts().plot(kind='bar')

In [ ]:
58+41+21

In [ ]:
scaler_X = MinMaxScaler()
all_X_normalized = scaler_X.fit_transform(all_X)


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    all_X, all_y, test_size=0.3, shuffle=True)  # , random_state=42

In [ ]:
# OverSampling

sm = RandomOverSampler(sampling_strategy='minority')
X_oversampled, y_oversampled = sm.fit_resample(all_X, all_y)

X_train, X_test, y_train, y_test = train_test_split(
    X_oversampled, y_oversampled, test_size=0.3, shuffle=True)  # 

clf = LogisticRegression().fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print(y_pred)
print(f1_score(y_test, y_pred, average=None))

In [ ]:
# OverSampling

sm = RandomOverSampler(sampling_strategy='minority')
X_oversampled, y_oversampled = sm.fit_resample(all_X, all_y)

X_train, X_test, y_train, y_test = train_test_split(
    X_oversampled, y_oversampled, test_size=0.3, shuffle=True)  # 

clf = LinearSVC().fit(X_train, y_train)
print(clf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print(y_pred)
print(f1_score(y_test, y_pred, average=None))

In [ ]:
len(y_test)

In [ ]:
pipe = Pipeline([('classifier' , RandomForestClassifier())])

param_grid = [
    { 
     'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    { 
     'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
    ,
   
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)
print("CV Score", clf.best_score_)
print(best_clf.score(X_test, y_test))
y_pred = best_clf.predict(X_test)
print(y_pred)
print(f1_score(y_test, y_pred, average=None))

# 0.6875 --> The best I got earlier 3 months ago
# ['A' 'A' 'B' 'A' 'C' 'C' 'B' 'B' 'C' 'B' 'B' 'C' 'A' 'C' 'C' 'A' 'B' 'B'
#  'C' 'C' 'B' 'C' 'A' 'B' 'A' 'C' 'B' 'B' 'C' 'A' 'A' 'C']
# [0.47058824 0.60869565 0.91666667]

In [ ]:
pipe = Pipeline([('classifier' , RandomForestClassifier())])

param_grid = [
    { 
     'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    { 
     'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
    ,
   
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

print(best_clf.score(X_test, y_test))
y_pred = best_clf.predict(X_test)
print(y_pred)
print(f1_score(y_test, y_pred, average=None))

In [ ]:
best_clf.best_params_

In [ ]:
for i in range(len(y_pred)):
    print("Predicted:", y_pred[i], " -- Reality",y_test[i])

In [ ]:
# Results:
# Using 'Credit Seeking' and removing 'D' and 'F' + MinMax + GridSearchCV: we got accuracy of 66.6% 
# Using 'Credit Seeking' and removing 'D' and 'F' + MinMax + LogisticRegression + SMOTE: we got accuracy of 58.3% 
# Using 'Credit Seeking' and removing 'D' and 'F' + MinMax + LogisticRegression: we got accuracy of 41.6% 
# Using 'Credit Seeking' and removing 'D' and 'F' + MinMax + DL (111,50,25,3): we got accuracy of 41.6% 

# Using 
# I have tried reducing the dimensions using PCA, and it turns out we can reduce to 45

In [ ]:
class MLPModel(pl.LightningModule):
    def __init__(self, n_features=111, n_hidden=45, n_output=3):
        super().__init__()
        # need to be (self) in order to be optimized and part of model
        self.model = nn.Sequential(
            nn.Linear(n_features, 128),
            nn.ReLU(),
             nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, n_output),
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)  # y_hat is (N, C) and y is (N)
        acc = FM.accuracy(F.softmax(y_hat, 1), y)
        self.log(
            "accuracy", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"loss": loss, "acc": acc}

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)  # y_hat is (N, C) and y is (N)
        acc = FM.accuracy(F.softmax(y_hat, 1), y)
        self.log(
            "accuracy", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"loss": loss, "acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

In [ ]:
le_y = preprocessing.LabelEncoder()
all_y = le_y.fit_transform(all_y)

X_train, X_test, y_train, y_test = train_test_split(
    all_X_normalized, all_y, test_size=0.2, shuffle=True, random_state=42)


X_train = torch.Tensor(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()


# params = {'shuffle': True,'num_workers': 0,'batch_size':256}
params = {"shuffle": True}

if __name__ == "__main__":

    dataset = torch.utils.data.TensorDataset(X_train.to(device), y_train.to(device))
    train_dataloader = torch.utils.data.DataLoader(dataset, **params)

    dataset = torch.utils.data.TensorDataset(X_test.to(device), y_test.to(device))
    test_dataloader = torch.utils.data.DataLoader(dataset)

    model = MLPModel()

trainer = pl.Trainer(max_epochs=50, progress_bar_refresh_rate=1)
trainer.fit(model, train_dataloader)
trainer.test(test_dataloaders=test_dataloader)
print("DONE")

In [ ]:
import pickle
import random
import sys

import matplotlib
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torchmetrics.functional as FM
from sklearn.metrics import classification_report, confusion_matrix 

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from torch import nn
from tqdm import tqdm

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import gc

from sklearn import preprocessing



# pl.utilities.seed.seed_everything(seed=2) # sets seed for pseudo-random number generators in: pytorch, numpy, python.random

In [ ]:
Clustered_Users.columns

In [ ]:
# all_X =  Clustered_Users[['# CrSk Attempts','# Nrml Attempts','% of CrSk',
#                     'Is CrSk','CrSk Time','Nrml Time','# CrSk Framesets','# Nrml Framesets']].to_numpy()

all_X = Clustered_Users[
    [
        "# CrSk Attempts",
        "% of CrSk",
        "Is CrSk",
        "CrSk Time",
        "# CrSk Framesets",
    ]
].to_numpy()
all_y = Clustered_Users["Letter Grade"].to_numpy()

scaler_X = MinMaxScaler()
all_X = scaler_X.fit_transform(all_X)
le_y = preprocessing.LabelEncoder()
all_y = le_y.fit_transform(all_y)
# scaler_y = StandardScaler()
# all_y = scaler_y.fit_transform(all_y)

X_train, X_test, y_train, y_test = train_test_split(
    all_X, all_y, test_size=0.2, shuffle=True, random_state=42
)

In [ ]:
clf = LinearSVC(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [ ]:
len(X_test), y_train.shape

In [ ]:
# Features:
# 1- Normalized Version of Credit Seeking
# 2- Normalized Version of Normal
# 3- Normalized Version of # of framesets in Credit Seeking
# 4- Normalized Version of # of framesets in Normal

In [ ]:
class MLPModelRegressor(pl.LightningModule):
    def __init__(self, n_features=2, n_hidden=128, n_output=3):
        super().__init__()
        # need to be (self) in order to be optimized and part of model
        self.model = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output),
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = FM.mean_squared_error(y_hat, y)
        self.log("MSE", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return {
            "loss": loss,
        }

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = FM.mean_squared_error(y_hat, y)
        self.log("MSE", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [ ]:
class MLPModel(pl.LightningModule):
    def __init__(self, n_features=5, n_hidden=50, n_output=5):
        super().__init__()
        # need to be (self) in order to be optimized and part of model
        self.model = nn.Sequential(
            nn.Linear(n_features, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, n_output),
        )

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)  # y_hat is (N, C) and y is (N)
        acc = FM.accuracy(F.softmax(y_hat, 1), y)
        self.log(
            "accuracy", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"loss": loss, "acc": acc}

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)  # y_hat is (N, C) and y is (N)
        acc = FM.accuracy(F.softmax(y_hat, 1), y)
        self.log(
            "accuracy", acc, on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"loss": loss, "acc": acc}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0001)

In [ ]:
# params = {'shuffle': True,'num_workers': 0,'batch_size':256}
params = {"shuffle": True}

if __name__ == "__main__":

    dataset = torch.utils.data.TensorDataset(X_train.to(device), y_train.to(device))
    train_dataloader = torch.utils.data.DataLoader(dataset, **params)

    dataset = torch.utils.data.TensorDataset(X_test.to(device), y_test.to(device))
    test_dataloader = torch.utils.data.DataLoader(dataset)

    model = MLPModel()

trainer = pl.Trainer(max_epochs=50, progress_bar_refresh_rate=1)
trainer.fit(model, train_dataloader)
trainer.test(test_dataloaders=test_dataloader)
print("DONE")

In [ ]:
# One Feature: % of Cr Seeking (Standridzed) with many hyperparameters, and the best MSE is 1.1431, Y = Final Score
# Two Feature: (Credit Seeking, Normal) (Standardized),                 and the best MSE is 1.220 , Y = Final Score
# Two Feature: (Credit Seeking, Normal) (Standardized), and the best MSE is 1.00264 , Y = (Midterm 1, Midterm 2, Final Exam)